# dim_produtos

Esse notebook é responsável por criar uma tabela com as informações básicas dos produtos do e-commerce.


## Configurações Iniciais

### Preparação do Ambiente PySpark

#### Instalando as Dependências

In [15]:
# Instala o Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Baixa o Apache Spark 3.4.1 com Hadoop 3 (versão estável e recente)
!wget -q https://archive.apache.org/dist/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz

# Descompacta o arquivo baixado
!tar xf spark-3.4.1-bin-hadoop3.tgz

# Instala a biblioteca findspark
!pip install -q findspark

#### Editando as Variáveis de Ambiente

In [16]:
import os

# Define a variável de ambiente do Java
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Define a variável de ambiente do Spark
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

from pyspark.sql import SparkSession

# Cria a SparkSession
spark = SparkSession.builder.master("local[*]").appName("E-commerce Dataset").getOrCreate()

### Importando as Bibliotecas

In [17]:
from pyspark.sql import functions as F

### Carregando os Dados

In [ ]:
df_products_dataset = (
    spark.read.csv(
        "/content/olist_products_dataset.csv",
        header=True,
        inferSchema=True
    )
    .withColumn(
        "product_category_name",
        F.regexp_replace(
           F.upper(F.col("product_category_name")),
           "_",
           " "
       )
    )
)

## Transformação dos Dados

In [ ]:
df_conversions = (
    df_products_dataset
    .withColumn(
        "product_weight_kg",
        F.col("product_weight_g") / 1000
    )
    .withColumn(
        "product_length_m",
        F.col("product_length_cm") / 100
    )
    .withColumn(
        "product_height_m",
        F.col("product_height_cm") / 100
    )
    .withColumn(
        "product_width_m",
        F.col("product_width_cm") / 100
    )
)

In [ ]:
df_final_products_dataset = (
    df_conversions
    .withColumn(
        "product_volume",
        F.col("product_length_m")
        * F.col("product_height_m")
        * F.col("product_width_m")
    )
    .select(
        "product_id",
        "product_category_name",
        "product_weight_kg",
        "product_length_m",
        "product_height_m",
        "product_width_m",
        "product_volume"
    )
)

## Salvamento dos Dados

In [20]:
(
    df_final_products_dataset
    .coalesce(1)
    .write
    .option("header", "true")
    .mode("overwrite")
    .csv("/transformed_data/dim_produtos")
)